This notebook explores word embeddings through the functionality of Gensim; we train new embeddings from a dataset of our own and compare with pre-trained Glove embeddings.

Before running, install gensim with:

`conda install gensim`


In [1]:
import re
from gensim.models import Word2Vec, KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.test.utils import datapath

First, let's train a new word2vec model on our data.

In [2]:
sentences=[]
filename="../data/wiki.10K.txt"
with open(filename) as file:
    for line in file:
        words=line.rstrip().lower()
        # this file is already tokenize, so we can split on whitespace
        # but first let's replace any sequence of whitespace (space, tab, newline, etc.) with single space
        words=re.sub("\s+", " ", words)
        sentences.append(words.split(" "))

In [3]:
model = Word2Vec(
        sentences,
        vector_size=100,
        window=5,
        min_count=2,
        workers=10)

In [12]:
my_trained_vectors = model.wv
model.wv
# save vectors to file if you want to use them later
#my_trained_vectors.save_word2vec_format('embeddings.txt', binary=False)

array([-2.6040468e+00,  4.6519479e-01,  1.3723152e+00,  2.4232153e-02,
        3.6689457e-01, -2.0964488e-01, -6.3129492e-02,  1.7304672e+00,
       -6.9785982e-01, -2.4084120e+00,  4.4760983e-02,  8.6913890e-01,
       -3.6593404e-01, -2.9760936e-01,  8.3753347e-01, -1.5732206e+00,
        1.0369208e+00, -2.8990591e-01, -1.4570014e+00, -2.8426056e+00,
        1.5044382e+00, -3.1966957e-01,  8.1602246e-01,  8.2970345e-01,
        5.3989303e-01, -8.2801718e-01,  1.1680928e+00,  8.2889128e-01,
       -2.7990193e+00, -1.0557247e+00,  1.8167909e+00,  1.1370176e+00,
        1.6069021e+00,  6.9361746e-01,  1.6337723e+00, -1.2583779e+00,
        1.2168442e+00, -2.5753796e+00, -1.3693327e+00,  1.1552408e+00,
       -8.0097437e-01, -1.4968573e+00,  8.7179160e-01, -5.0303859e-01,
        2.8146556e-01, -1.3122885e+00, -7.6302958e-01,  1.0624727e+00,
        5.2453274e-01, -1.2822746e+00, -5.0274181e-01,  3.5793233e-01,
       -1.7118424e+00, -1.5503944e+00,  4.3970987e-01,  3.2836959e-01,
      

In [6]:
my_trained_vectors.most_similar("actor", topn=10)

[('actress', 0.9563195109367371),
 ('musician', 0.9091904759407043),
 ('composer', 0.9039162993431091),
 ('producer', 0.8941104412078857),
 ('writer', 0.8919810056686401),
 ('artist', 0.8903812170028687),
 ('dancer', 0.8714725971221924),
 ('singer', 0.864169180393219),
 ('pianist', 0.8626704812049866),
 ('journalist', 0.8625409007072449)]

Let's load in vectors that have already been trained on a much bigger dataset. [Glove vectors](https://nlp.stanford.edu/projects/glove/) are trained using a different method than word2vec, but results in vectors that can be read in by Gensim.  Here we'll use a 100-dimensional model trained on 6B words (from Wikipedia and news), but bigger models are also available.

In [5]:
# First we have to convert the Glove format into w2v format; this creates a new file
glove_file="../data/glove.6B.100d.100K.txt"
glove_in_w2v_format="../data/glove.6B.100d.100K.w2v.txt"
_ = glove2word2vec(glove_file, glove_in_w2v_format)

<ipython-input-5-86a990e6d4aa>:4: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  _ = glove2word2vec(glove_file, glove_in_w2v_format)


In [7]:
glove = KeyedVectors.load_word2vec_format("../data/glove.6B.100d.100K.w2v.txt", binary=False)

In [11]:
glove.most_similar("actor", topn=10)

[('actress', 0.8580666184425354),
 ('comedian', 0.795758843421936),
 ('starring', 0.7920297384262085),
 ('starred', 0.7582033276557922),
 ('actors', 0.7394535541534424),
 ('filmmaker', 0.7349801063537598),
 ('screenwriter', 0.7342271208763123),
 ('film', 0.6941469311714172),
 ('movie', 0.6924506425857544),
 ('comedy', 0.6884662508964539)]

`most_similar` allows for vector arithmetic (as the average value of the input positive/negative vectors, where negative vectors are first multiplied by -1).  Play around with this function to discover other analogies that have been learned in this representation.

In [13]:
# one + two = three + ?
one="man"
two="king"
three="woman"

one="paris"
two="france"
three="berlin"

glove.most_similar(positive=[two, three], negative=[one], topn=5)

[('germany', 0.892362117767334),
 ('austria', 0.7597677111625671),
 ('poland', 0.7425415515899658),
 ('denmark', 0.7360999584197998),
 ('german', 0.6986513733863831)]

We can also evaluate the quality of the learned vectors through an intrinsic evaluation comparing to human judgments in the wordsim 353 dataset.

In [14]:
glove.evaluate_word_pairs(datapath('wordsim353.tsv'))

((0.5483502278951546, 4.235096667699337e-29),
 SpearmanrResult(correlation=0.5327354323238274, pvalue=2.8654146580558905e-27),
 0.0)

In [15]:
my_trained_vectors.evaluate_word_pairs(datapath('wordsim353.tsv'))

((0.38471142639602574, 1.0135344149176915e-13),
 SpearmanrResult(correlation=0.39151377703291373, pvalue=3.399532939047208e-14),
 1.41643059490085)